In [10]:
#|default_exp img_sen2

# Sentinel 2

In [11]:
#|hide
from nbdev.showdoc import *

In [12]:
#|hide
#|export

import geopandas as gpd
import seaborn as sns
from shapely.geometry import Point
import json
import ee
import geemap
from datetime import datetime

ee.Authenticate()
ee.Initialize(project='ee-reutkeller')

from gee_projects import util_func as utils
from gee_projects import const_vars as const
from gee_projects import mask_s2 as mask_s2

sentinel-2 class

In [13]:
#|export

class access_s2_image():
  
  def __init__(self,
               path_to_geometry : str , # path to geometry file, can be .shp or .gpkg . example : r"D:\git\gee_maps\nbs\geometry\test_bbox.gpkg"
               crs : str , #coordinate reference system to work with
               start_date : str , # Start date for searching images. , 
               end_date : str , # End date for searching images.
               ):
    
    self.crs = crs
    self.fc = utils.gdf_to_featureCollection(
      utils.read_gdf(geometry_path=path_to_geometry,crs=self.crs))
    self.start_date=start_date
    self.end_date=end_date
    
    
    self.fc = utils.gdf_to_featureCollection(utils.read_gdf
                                        (geometry_path=path_to_geometry,crs=self.crs))
    
    #pre process s2
    s2_sr_cld_col_eval = mask_s2.get_s2_sr_cld_col(self.fc, self.start_date, self.end_date)

    self.s2_sr = (s2_sr_cld_col_eval.map(mask_s2.add_cld_shdw_mask)
             .map(mask_s2.apply_cld_shdw_mask))

    #get list of available dates of the image collection

    self.available_dates = utils.list_dates(self.s2_sr)
    

    

    

In [14]:
extractor = access_s2_image(path_to_geometry=r"D:\git\gee_maps\nbs\geometry\test_bbox.gpkg", 
                            crs = "4326",
                            start_date = '2023-05-06',
                            end_date = '2023-10-08')

coll= extractor.s2_sr
test=extractor.available_dates
test

AttributeError: module 'gee_projects.util_func' has no attribute 'list_dates'

In [ ]:
def list_dates(img_coll):
  # Function to extract dates from image collection
  def extract_dates(img):
      date = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd')
      return ee.Feature(None, {'date': date})

  # Map the function over the image collection
  dates_list = coll.map(extract_dates).aggregate_array('date').getInfo()
  print(dates_list)

In [ ]:
# Function to extract dates from image collection
def extract_dates(img):
    date = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd')
    return ee.Feature(None, {'date': date})

# Map the function over the image collection
dates_list = coll.map(extract_dates).aggregate_array('date').getInfo()
print(dates_list)

In [ ]:
def find_closest_date(user_date, date_list):
    # Convert user-provided date string to datetime object
    user_datetime = datetime.strptime(user_date, '%Y-%m-%d')

    # Convert the list of date strings to datetime objects
    date_objects = [datetime.strptime(date, '%Y-%m-%d') for date in date_list]

    # Find the closest date in the list
    closest_date = min(date_objects, key=lambda date: abs(date - user_datetime))

    # Convert the closest date back to string format
    closest_date_str = closest_date.strftime('%Y-%m-%d')

    return closest_date_str

In [ ]:
find_closest_date('2023-10-19', dates_list)

In [ ]:
Map = geemap.Map()

Map.add_basemap("SATELLITE")

# Viz = {'min': 0, 'max': 5000, 'palette': [    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
#                                          '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
#                                          '012E01', '011D01', '011301']}

Map.addLayer(coll.first().select(['B4','B3','B2']), {}, "img1")

Map.centerObject(extractor.fc)
Map

In [ ]:
dates_list